<a href="https://colab.research.google.com/github/WitneySouza/Pesquisa-Operacional/blob/main/ProjetoSupermercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Supermercado (OTIMIZAÇÃO)

> Alunos: Witney Reinande de Souza Araujo, Vicente Nascimento Souza.


## Projeto de Pesquisa Operacional


> Considerando:


- A planilha (link) com o preço de 87 produtos em 10 supermercados diferentes (Obs. estes
preços foram gerados aleatoriamente com objetivo didático e não refletem a realidade
destes supermercados citados);
- A lista de hotéis e suas respectivas distâncias de cada um dos supermercados (Aba
Hotéis);
- A lista (L1) de intenção de compras com os seguintes produtos:
1,4,5,15,23,35,45,47,59,60 e 80, e que deseja-se minimizar o gasto para a realização das
compras, pede-se:
1. Modele o problema matematicamente;
2. Qual o melhor supermercado para compras para os hóspedes de cada um dos
hotéis, sabendo que o custo de deslocamento é de R$0,70/km ?

3. Caso seja possível comprar alguns produtos em um supermercado e outros
produtos em outro(s) supermercado(s), qual seria a melhor escolha de cada um dos
hóspedes?
4. Desenvolva uma interface gráfica em que o usuário selecione uma lista de produtos
desejados, e qual hotel está hospedado, e então, o sistema deverá fornecer qual a
melhor opção de compra.
Para realização deste trabalho, você deverá modelar o problema como um problema de
otimização linear, sujeito às seguintes restrições:
- O usuário deverá comprar todos os itens em somente um supermercado (questão 2);
- Deve ser computado o preço praticado por serviços de entrega (R$1,00/km) com base na
distância entre o hotel e o supermercado escolhido (Questão 2);
- Todos os itens escolhidos pelo usuário devem ser comprados;

- Os itens sempre estarão disponíveis para as compras;

- O usuário poderá comprado mais de um item de cada produto;

- O usuário só tem disponível R$200 para a compra;

- O usuário poderá comprar os itens em em supermercados diferentes (questão 3);

>Itens extra:

- Faça uma análise de sensibilidade alterando a composição de produtos na cesta
- Considere diferentes disponibilidades (R$200, R$300, R$400 e R$500)
> Planilha:
https://docs.google.com/spreadsheets/d/1k1Lcd6vWT8sO3vrMx-rKzyE8U2RtPoPF9HZyl8taIv0/edit?usp=sharin




> MODELAGEM DE OTIMIZAÇÃO

Temos um grafo bipartido G(V;E), onde V = V1 ∩ V2 e V = {h1,h2...,h14,s1,s2,...,s10} é o conjunto de vértices e E = {(hi,sj) : hi ∈ V1 e sj ∈ V2. i > j} é o conjunto de arestas que associa cada hotel i a um supermercado j. 

Temos também o grafo bipartido C(I;P), onde I = {p1,p2,...,p87,s1,s2,...,sj} é o conjunto de vértices e P = {(pk,sj) : pk ∈ I1 e pj ∈ I2. k > j} é o conjunto de arestas que associa cada produto k a um supermercado j. 

- Variáveis:
1. h_ij = variável binária. ( 1 se o percurso for usado, 0 caso contrário)
2. c_kj = matriz com o preço de cada produto k em cada supermercado j.
3. d_ij = matriz das distâncias entre os hotéis e supermercados.


- Função Objetivo: Min \sum_{j=1}^{14} c_{1j} + c_{4j} + c_{5j} + c{15j} + c_{23j} + c_{35j} + c_{45j} + c{47j} + c_{59j} + c_{60j} + c_{80j} + \sum_{i=1}^{14} \sum_{j=1}^{10} d_ij*h_ij*0,7

- Restrições:
sum_{j=1}^{14} h_ij = 1
sum{j=1}^{14} c_{1j} + c_{4j} + c_{5j} + c{15j} + c_{23j} + c_{35j} + c_{45j} + c{47j} + c_{59j} + c_{60j} + c_{80j} ≤ 200





In [ ]:
!pip install gurobipy
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

hotel = 'Hotel1'


supermercados = {
    'Supermercado1',
    'Supermercado2'
}

produtos = {
    'Produto1',
    'Produto2'
}

valoresProdutos = {
    ('Produto1', 'Supermercado1'): 23,
    ('Produto2', 'Supermercado1'): 40,
    ('Produto1', 'Supermercado2'): 23,
    ('Produto2', 'Supermercado2'): 40,
}

distances = {
    ('Supermercado1', 'Hotel1'): 23,
    ('Supermercado2', 'Hotel1'): 22,
}

comprou = {
    'Supermercado1',
    'Supermercado2'
}

# Model
m = gp.Model("Otimizacao de compra")

prd = m.addVars(comprou, name='prd', vtype=GRB.BINARY)

m.setObjective(
    (sum((valoresProdutos[p, s] + (distances[s, hotel] * 2)) * prd[s] for p in produtos for s in supermercados)),
    GRB.MINIMIZE)

m.addConstr(sum(prd[s] for s in supermercados) == 1)

m.addConstrs((gp.quicksum(valoresProdutos[p, s] + (distances[s, hotel] * 2) * prd[s] for p in produtos) <= 200 for s in
              supermercados), "_")


def printSolution():
    print('\nBuy:')
    for s in supermercados:
        if prd[s].X > 0:
            print('%s' % s)
        else:
            print('No solution')


# Solve
m.optimize()
printSolution()


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x2699abdf
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [2e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 151.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 151 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.510000000000e+02, best bound 1.510000000000e+02, gap 0.0000%

Buy:
Supermercado2
No solution
